In [1]:
!sudo pip install fastapi loguru

In [2]:
# !sudo  curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

In [3]:
# !sudo pip install ngrok-api

In [2]:
import asyncio
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

In [5]:
# !sudo pip uninstall pyngrok
# from pyngrok import ngrok

# # Construct the API client
# # client = ngrok.Client("2miMoGd29rsmymap1vUca35jysd_2Li6vyyBXQmtEvhTFwNV6")
# import os
# os.system('ngrok authtoken 2miMoGd29rsmymap1vUca35jysd_2Li6vyyBXQmtEvhTFwNV6')
# url = ngrok.connect(port = 80)
# print(url)

In [6]:
# !ngrok config add-authtoken 1yinXFJICgluDFqwlX1WDdJVWAS_7aeqVsZNd6cs8EmLSeiYC
# !ngrok http --url=correctly-awake-dogfish.ngrok-free.app 80

In [3]:
class Request(BaseModel):
    question: str


class Response(BaseModel):
    answer: str
    class_1: str
    class_2: str

app = FastAPI()


@app.get("/")
def index():
    return {"text": "Интеллектуальный помощник оператора службы поддержки."}

    
@app.post("/predict")
async def predict_sentiment(request: Request):
    question_embedding = model.encode(request.question)
    similarities = cos_sim(question_embedding, faq_embeddings)
    anwer_data = faq.iloc[similarities.argmax().item()]
    response = Response(
        answer=anwer_data['Ответ из БЗ'],
        class_1=anwer_data['Классификатор 1 уровня'], # Классификатор оценивается опционально; при отсутствии можно задать константное значение.
        class_2=anwer_data['Классификатор 2 уровня'], # Классификатор оценивается опционально; при отсутствии можно задать константное значение.
    )
    return response
# :4040  
if __name__ == "__main__":
    host = "127.0.0.1" # Сконфигурируйте host согласно настройкам вашего сервера.
    config = uvicorn.Config(app, host=host, port=80)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())


INFO:     Started server process [9303]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 13] error while attempting to bind on address ('127.0.0.1', 80): permission denied
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [8]:
!sudo pip install ngrok-api
from contextlib import asynccontextmanager
from os import getenv
import asyncio
import ngrok
import uvicorn
from fastapi import FastAPI
from loguru import logger

NGROK_AUTH_TOKEN = getenv("NGROK_AUTH_TOKEN", "")
NGROK_EDGE = getenv("NGROK_EDGE", "edge:edghts_")
APPLICATION_PORT = 4040

# ngrok free tier only allows one agent. So we tear down the tunnel on application termination
@asynccontextmanager
async def lifespan(app: FastAPI):
    logger.info("Setting up Ngrok Tunnel")
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    ngrok.forward(
        addr=APPLICATION_PORT,
        labels=NGROK_EDGE,
        proto="labeled",
    )
    yield
    logger.info("Tearing Down Ngrok Tunnel")
    ngrok.disconnect()


app = FastAPI(lifespan=lifespan)

@app.get("/")
async def root():
    return {"message": "Hello World"}

if __name__ == "__main__":
    host = "127.0.0.1" # Сконфигурируйте host согласно настройкам вашего сервера.
    config = uvicorn.Config(app, host=host, port=4041)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())

INFO:     Started server process [9202]
INFO:     Waiting for application startup.
2024-09-28 21:09:24.820 | INFO     | __main__:lifespan:17 - Setting up Ngrok Tunnel
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 693, in lifespan
    async with self.lifespan_context(app) as maybe_state:
  File "/usr/lib/python3.10/contextlib.py", line 199, in __aenter__
    return await anext(self.gen)
  File "/tmp/ipykernel_9202/127282004.py", line 18, in lifespan
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
AttributeError: module 'ngrok' has no attribute 'set_auth_token'

ERROR:    Application startup failed. Exiting.
